## Raster Masking
This code is made simply to mask any raster using polygon

In [ ]:
# Installing required libraries
# Skip if you have installed these libs
%pip install rasterio geopandas

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd

In [ ]:
# Input data
# This path is for files in the same folder as the code
polypath = "zwolle_gementee.geojson" # Use any format recognized by geopandas
rasterpath = "zwolle_precipitation.tiff" # Use any format recognized by rasterio

In [ ]:
# VECTOR
polygon_data = gpd.read_file(polypath)
# Transform into EPSG:4326, comment out if you use other CRS
if polygon_data.crs != "EPSG:4326":
    polygon_data = polygon_data.to_crs("EPSG:4326")

In [ ]:
# RASTER
with rasterio.open(rasterpath) as src:
    # Masking the raster with the polygon
    out_image, out_transform = mask(src, polygon_data.geometry, crop=True, all_touched=True, invert=False)
    
    # Update the metadata
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "nodata": src.nodata
    })

In [ ]:
# Saving
maskedrasterpath = "masked_raster.tif" # Changed to your desired output path
with rasterio.open(maskedrasterpath, "w", **out_meta) as dest:
    dest.write(out_image)
    
print("Saved to:", maskedrasterpath)